In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental yfiles-jupyter-graphs neo4j PyPDF2 json-repair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 779.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from neo4j import GraphDatabase
import os
from langchain_community.graphs import Neo4jGraph
import PyPDF2
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.text_splitter import TokenTextSplitter
from langchain.graphs import Neo4jGraph
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
#import streamlit as st

In [ ]:
NEO4J_URI="neo4j+s://########.databases.neo4j.io:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="#######################"

In [ ]:
os.environ['OPENAI_API_KEY'] = '###########################'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [ ]:
graph = Neo4jGraph()
driver = GraphDatabase.driver(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)


In [ ]:
graph = Neo4jGraph()

In [ ]:
# Define the file paths for your 4 PDFs
pdf_files = [
    '/content/Paper1.pdf',  # Replace with the path to your first PDF
    '/content/Paper2.pdf',  # Replace with the path to your second PDF
    '/content/Paper3.pdf',  # Replace with the path to your third PDF
    '/content/Paper4.pdf'   # Replace with the path to your fourth PDF
]

In [ ]:
# Initialize a list to hold all the documents
all_documents = []

# Loop through each PDF file
for file_path in pdf_files:
    pdf_reader = PyPDF2.PdfReader(open(file_path, "rb"))

    # Extract text from each page of the PDF and create Document objects
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        all_documents.append(Document(page_content=text))


In [ ]:
# Check the first document
print(all_documents[0].page_content)

Citation: Weaver, D.F. Thirty Risk
Factors for Alzheimer’s Disease
Unified by a Common
Neuroimmune–Neuroinflammation
Mechanism. Brain Sci. 2024 ,14, 41.
https://doi.org/10.3390/brainsci
14010041
Academic Editors: Junhui Wang,
Hongxing Wang and Jing Sun
Received: 29 November 2023
Revised: 27 December 2023
Accepted: 30 December 2023
Published: 31 December 2023
Copyright: ©2023 by the author.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed under the terms and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
brain sciences 
Review
Thirty Risk Factors for Alzheimer’s Disease Unified by a
Common Neuroimmune–Neuroinflammation Mechanism
Donald F. Weaver
Krembil Research Institute, University Health Network, Departments of Medicine, Chemistry, Pharmaceutical
Sciences, University of Toronto, Toronto, ON M5T 0S8, Canada; donald.weaver@uhnresearch.ca
Abstract: One of the major obstacles confrontin

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(all_documents)
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")
llm_transformer = LLMGraphTransformer(llm=llm)

<ipython-input-14-0826f6001701>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")


In [ ]:
#graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [ ]:
file_path = '/content/graph_documents.txt'  # Adjust this path if the file is located elsewhere

# Read the file content
with open(file_path, 'r') as file:
    graph_documents_json = file.read()
import json

# Convert JSON string back to a Python object (list of graph documents)
graph_documents = json.loads(graph_documents_json)

# Check the loaded graph documents
print(graph_documents)


[{'nodes': [{'id': 'tau containing neurofibrillary tangles', 'type': 'Finding', 'properties': {}}, {'id': 'age at onset of Alzheimer disease', 'type': 'Characteristic', 'properties': {}}, {'id': 'amyloid plaques', 'type': 'Finding', 'properties': {}}, {'id': 'dementia', 'type': 'Condition', 'properties': {}}, {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}}, {'id': 'Apo lipoprotein E', 'type': 'Gene', 'properties': {}}], 'relationships': [{'source': {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}}, 'target': {'id': 'dementia', 'type': 'Condition', 'properties': {}}, 'type': 'IS_CAUSE_OF', 'properties': {}}, {'source': {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}}, 'target': {'id': 'amyloid plaques', 'type': 'Finding', 'properties': {}}, 'type': 'HAS_PATHOLOGIC_FINDINGS', 'properties': {}}, {'source': {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}}, 'target': {'id': 'tau containing neurofibrillary tangles', 'type': 'Fin

In [ ]:
graph_documents

[{'nodes': [{'id': 'tau containing neurofibrillary tangles',
    'type': 'Finding',
    'properties': {}},
   {'id': 'age at onset of Alzheimer disease',
    'type': 'Characteristic',
    'properties': {}},
   {'id': 'amyloid plaques', 'type': 'Finding', 'properties': {}},
   {'id': 'dementia', 'type': 'Condition', 'properties': {}},
   {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}},
   {'id': 'Apo lipoprotein E', 'type': 'Gene', 'properties': {}}],
  'relationships': [{'source': {'id': 'Alzheimer disease',
     'type': 'Disease',
     'properties': {}},
    'target': {'id': 'dementia', 'type': 'Condition', 'properties': {}},
    'type': 'IS_CAUSE_OF',
    'properties': {}},
   {'source': {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}},
    'target': {'id': 'amyloid plaques', 'type': 'Finding', 'properties': {}},
    'type': 'HAS_PATHOLOGIC_FINDINGS',
    'properties': {}},
   {'source': {'id': 'Alzheimer disease', 'type': 'Disease', 'properties': {}}

In [ ]:
import json

# Assuming `graph_documents` is the output you want to save
# Convert graph_documents to JSON string for better readability
#graph_documents_json = json.dumps([doc.dict() for doc in graph_documents], indent=4)

# Save to a .txt file
#with open('graph_documents.txt', 'w') as file:
#    file.write(graph_documents_json)

#print("Graph documents saved successfully to 'graph_documents.txt'")


Graph documents saved successfully to 'graph_documents.txt'


In [ ]:
graph = Neo4jGraph(
    url="neo4j+s://##############.databases.neo4j.io:7687",
    username="neo4j",
    password="#######################"
)

# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Create the vector index from the existing graph
vector_index = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    graph=graph,
    index_name='vector_index',
    node_label="Embeddable",
    text_node_properties=['definition', 'term', 'clause'],
    embedding_node_property='embedding',
)

<ipython-input-17-9d53ef4eeb8e>:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()


In [ ]:
# Set up the LLM (Language Model)
llm = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo-16k")

# Create a retrieval-based QA chain using the vector index
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever(search_type="similarity"),
    return_source_documents=True
)


In [ ]:
# Define the chatbot function
def alzheimers_chatbot(query):
    result = qa_chain({"query": query})
    answer = result["result"]
    source_docs = result["source_documents"]
    return answer, source_docs

In [ ]:
# Example usage
user_query = "What are the risk factors for Alzheimer's disease?"
response, sources = alzheimers_chatbot(user_query)

print("Response:", response)
print("\nSources:")
for doc in sources:
    print(f"- {doc.metadata['source']}")


<ipython-input-31-b44b81f55b6c>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": query})


Response: The risk factors for Alzheimer's disease include age, family history, genetics, certain medical conditions (such as high blood pressure, diabetes, and heart disease), head injuries, and lifestyle factors (such as smoking, lack of physical exercise, and poor diet).

Sources:


In [ ]:
!pip install streamlit
!pip install pyngrok
import streamlit as st

In [ ]:

# Function to initialize vector index and QA chain
def initialize_qa_chain():
    # Initialize the OpenAI embeddings
    embeddings = OpenAIEmbeddings()

    # Create the vector index from the existing graph
    vector_index = Neo4jVector.from_existing_graph(
        embedding=embeddings,
        graph=graph,
        index_name='vector_index',
        node_label="Embeddable",
        text_node_properties=['definition', 'term', 'clause'],
        embedding_node_property='embedding',
    )

    # Set up the LLM (Language Model)
    llm = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo-16k")

    # Create a retrieval-based QA chain using the vector index
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_index.as_retriever(search_type="similarity"),
        return_source_documents=True
    )


In [ ]:
qa_chain = initialize_qa_chain()


In [ ]:
# Streamlit UI
st.title("Alzheimer's Disease Chatbot")

2024-09-10 04:31:04.790 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 04:31:04.939 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-10 04:31:04.955 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
# User input for the query
user_query = st.text_input("Enter your question about Alzheimer's disease:")


2024-09-10 04:31:15.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 04:31:15.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 04:31:15.980 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 04:31:15.982 Session state does not function when running a script without `streamlit run`
2024-09-10 04:31:15.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 04:31:15.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
if user_query:
    with st.spinner("Retrieving answer..."):
        # Call the chatbot function
        response, sources = qa_chain({"query": user_query})

        # Display the response
        st.subheader("Response:")
        st.write(response['result'])

        # Display the sources
        st.subheader("Source Documents:")
        for doc in response['source_documents']:
            st.write(f"- {doc.metadata['source']}")


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Replace "your_authtoken" with the actual token from ngrok dashboard
!ngrok authtoken ##############


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st

st.title("Alzheimer's RAG Chatbot")

query = st.text_input("Enter your question:")

if st.button("Submit"):
    if query:
        response, sources = alzheimers_chatbot(query)  # Replace with your chatbot function
        st.write("Response:", response)
        st.write("\nSources:")
        for doc in sources:
            st.write(f"- {doc.metadata['source']}")


Overwriting app.py


In [ ]:
#from pyngrok import ngrok

# Create the Streamlit tunnel
#public_url = ngrok.connect(port='8501', proto='http')

# Run the Streamlit app
#!streamlit run app.py &

# Output the public URL
#print(f"Public URL for your Streamlit app: {public_url}")

from pyngrok import ngrok

# Authenticate your ngrok account
!ngrok authtoken ######################################

# Create an HTTP tunnel for Streamlit on port 8501
public_url = ngrok.connect(8501, "http")

# Print the public URL to access the Streamlit app
print(f"Public URL for your Streamlit app: {public_url}")

# Run the Streamlit app in the background
!streamlit run app.py &


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL for your Streamlit app: NgrokTunnel: "https://25cf-34-173-233-29.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.173.233.29:8501

2024-09-10 05:20:07.637 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 590, in code_to_exec
    exec(code, module.__dict__)
  File "/content/app.py", line 9, in <module>
    response, sources = alzheimers_chatbot(query)  # Replace with your chatbot function
NameError: name 'alzheimers_chatbot' is not defined


In [ ]:
%%writefile app.py
import streamlit as st
from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.neo4j_vector import Neo4jVector

# Define the Neo4j graph connection
graph = Neo4jGraph(
    url="neo4j+s://###c0.databases.neo4j.io:7687",
    username="neo4j",
    password="jc###################
)

# Function to initialize vector index and QA chain
def initialize_qa_chain():
    # Initialize the OpenAI embeddings
    embeddings = OpenAIEmbeddings()

    # Create the vector index from the existing graph
    vector_index = Neo4jVector.from_existing_graph(
        embedding=embeddings,
        graph=graph,
        index_name='vector_index',
        node_label="Embeddable",
        text_node_properties=['definition', 'term', 'clause'],
        embedding_node_property='embedding',
    )

    # Set up the LLM (Language Model)
    llm = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo-16k")

    # Create a retrieval-based QA chain using the vector index
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_index.as_retriever(search_type="similarity"),
        return_source_documents=True
    )

    return qa_chain

# Initialize the QA chain once
qa_chain = initialize_qa_chain()

# Define the chatbot function
def alzheimers_chatbot(query):
    result = qa_chain({"query": query})
    answer = result["result"]
    source_docs = result["source_documents"]
    return answer, source_docs

#######################################################################################################
# Streamlit UI
#st.title("Alzheimer's Disease RAG Chatbot")

# User input for the query
#query = st.text_input("Enter your question about Alzheimer's disease:")

# Run the chatbot when the user submits a query
#if st.button("Submit"):
 #   if query:
  #      response, sources = alzheimers_chatbot(query)

        # Display the response
 #       st.subheader("Response:")
#        st.write(response)

        # Display the sources
#        st.subheader("Source Documents:")
 #       for doc in sources:
 #           st.write(f"- {doc.metadata['source']}")
########################################################################################################

# Streamlit UI
st.title("Alzheimer's Disease RAG Chatbot")

# Initialize the session state to store the chat history
if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = []

# User input for the query
query = st.text_input("Ask your question about Alzheimer's disease:")

# Run the chatbot when the user submits a query
if st.button("Submit"):
    if query:
        # Call the chatbot function and get the response
        response, sources = alzheimers_chatbot(query)

        # Add the user's question and the model's response to the chat history
        st.session_state["chat_history"].append({"query": query, "response": response})

# Display the conversation history
st.subheader("Conversation:")
for chat in st.session_state["chat_history"]:
    st.write(f"**You:** {chat['query']}")
    st.write(f"**Chatbot:** {chat['response']}")

# Optionally, display sources
if st.checkbox("Show Sources"):
    st.subheader("Source Documents:")
    for doc in sources:
        st.write(f"- {doc.metadata['source']}")

Writing app.py


In [ ]:
!pip install streamlit pyngrok neo4j openai
!ngrok authtoken ##################################


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully u

In [ ]:
!streamlit run app.py &
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"Public URL for your Streamlit app: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.41.254:8501

